In [ ]:
from pyscf_util.misc.mole import get_mol
from pyscf_util.MeanField.scf import kernel as scf
from pyscf.tools import mo_mapping
import numpy

In [ ]:
molecule = get_mol(
    xyz="""
C1 0.         0.         0.002402  
O2 0.         0.         2.12996496
""",
    spin=0,
    charge=0,
    basis="cc-pvdz",
    unit="Bohr",
)

In [ ]:
for label in molecule.ao_labels(None):
    if label[2] == '2p' and label[3] == 'z':
        print(label)

In [ ]:
idx = molecule.search_ao_label(['C1.*'])
print(idx)

In [ ]:
def classify_orbitals(mo_energy, threshold=1e-5, verbose=False):
    
    # 初始化分组
    groups = []
    current_group = [mo_energy[0]]
    
    # 根据阈值对能量值进行分组
    for energy in mo_energy[1:]:
        if energy - current_group[-1] <= threshold:
            current_group.append(energy)
        else:
            groups.append(current_group)
            current_group = [energy]
    groups.append(current_group)
    
    principle_number = {
        "s": 1,
        "p": 2,
        "d": 3,
        "f": 4,
        "g": 5,
        "h": 6,
        "i": 7
    }
    
    # 确定每组的轨道类型
    orbital_types = []
    for group in groups:
        count = len(group)
        if count == 1:
            orbital_types.extend([f'{principle_number["s"]}s'] * count)
            principle_number['s'] += 1
        elif count == 3:
            orbital_types.extend([f'{principle_number["p"]}p'] * count)
            principle_number['p'] += 1
        elif count == 5:
            orbital_types.extend([f'{principle_number["d"]}d'] * count)
            principle_number['d'] += 1
        elif count == 7:
            orbital_types.extend([f'{principle_number["f"]}f'] * count)
            principle_number['f'] += 1
        elif count == 9:
            orbital_types.extend([f'{principle_number["g"]}g'] * count)
            principle_number['g'] += 1
        elif count == 11:
            orbital_types.extend([f'{principle_number["h"]}h'] * count)
            principle_number['h'] += 1
        elif count == 13:
            orbital_types.extend([f'{principle_number["i"]}i'] * count)
            principle_number['i'] += 1
        else:
            orbital_types.extend(['unknown'] * count)
    
    # 打印结果
    if verbose:
        print("能量\t\t轨道类型")
        for energy, orbital_type in zip(mo_energy, orbital_types):
            print(f"{energy:15.6f}\t\t{orbital_type}")
    
    return orbital_types

In [ ]:
from pyscf_util._atmMinCASOrb._orb_loader import LoadAtmHFOrbAllInfo, LoadAtmHFOrbGivenType

info = LoadAtmHFOrbAllInfo("C", 0, "cc-pvdz", with_sfx2c=True)

classify_orbitals(info['mo_energy'], verbose=True)

In [ ]:
info = LoadAtmHFOrbAllInfo("N", 0, "cc-pvdz", with_sfx2c=True)

In [ ]:
info = LoadAtmHFOrbAllInfo("O", 0, "cc-pvdz", with_sfx2c=True)

In [ ]:
info = LoadAtmHFOrbAllInfo("F", 0, "cc-pvdz", with_sfx2c=True)

In [ ]:
info = LoadAtmHFOrbAllInfo("Si", 0, "unc-cc-pvdz", with_sfx2c=True)
classify_orbitals(info['mo_energy'], verbose=True)
print(LoadAtmHFOrbGivenType("Si", 0, "unc-cc-pvdz", with_sfx2c=True, orb_symbol=["2p", "3p"]))

In [ ]:
info = LoadAtmHFOrbAllInfo("H", 0, "unc-cc-pvdz", with_sfx2c=True)
classify_orbitals(info['mo_energy'], verbose=True)